In [13]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [14]:
#Drop the non-beneficial ID columns, 'EIN' and 'NAME', 

application_df= application_df.drop(["EIN", "NAME","AFFILIATION", "USE_CASE", "SPECIAL"], axis=1)
application_df=application_df.dropna()
#application_df=application_df[application_df["INCOME_AMT"] > "0"]
application_df.head()

,APPLICATION_TYPE,CLASSIFICATION,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,C1000,Association,1,0,N,5000,1
1,T3,C2000,Co-operative,1,1-9999,N,108590,1
2,T5,C3000,Association,1,0,N,5000,0
3,T3,C2000,Trust,1,10000-24999,N,6692,1
4,T3,C1000,Trust,1,100000-499999,N,142590,1


In [15]:
#APPLICATION COUNT 
application_counts= application_df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are less than ...?

replace_application = list(application_counts[application_counts < 100].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [16]:
# Generate our categorical variable lists
application_cat=application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'CLASSIFICATION',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [17]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
#  YOUR CODE GOES HERE
encode_df=pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [18]:
# Merge one-hot encoded features and drop the originals

application_df=application_df.merge(encode_df, left_index=True, right_index=True)
application_df=application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
# Split our preprocessed data into our features and target arrays

X=application_df.drop(["IS_SUCCESSFUL"],1).values
y=application_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 1000
hidden_nodes_layer2 = 800
hidden_nodes_layer3= 600
hidden_nodes_layer4=400
hidden_nodes_layer5=200

nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))
# fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))
#fifth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 1000)              99000     
_________________________________________________________________
dense_7 (Dense)              (None, 800)               800800    
_________________________________________________________________
dense_8 (Dense)              (None, 600)               480600    
_________________________________________________________________
dense_9 (Dense)              (None, 400)               240400    
_________________________________________________________________
dense_10 (Dense)             (None, 200)               80200     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 201       
Total params: 1,701,201
Trainable params: 1,701,201
Non-trainable params: 0
____________________________________________

In [22]:
# Compile the model

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=5,
    save_weights_only=True,
    save_freq=1000)

In [24]:
# Train the model

fit_model=nn.fit(X_train, y_train, epochs=50,callbacks=[cp_callback])

Epoch 1/50
   32/25724 [..............................] - ETA: 26s - loss: 0.6635 - acc: 0.6250
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 1024/25724 [>.............................] - ETA: 26s - loss: 0.6928 - acc: 0.5342
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 2048/25724 [=>............................] - ETA: 25s - loss: 0.6940 - acc: 0.5264
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 3072/25724 [==>...........................] - ETA: 24s - loss: 0.6964 - acc: 0.5124
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 4096/25724 [===>..........................] - ETA: 23s - loss: 0.6957 - acc: 0.5127
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 5152/25724 [=====>........................] - ETA: 25s - loss: 0.6951 - acc: 0.5136
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 6176/25724 [======>.......................] - ETA: 23s - loss: 0.6947 - acc: 0.5136
Epoch 00001: saving model to checkpoints/weights.01.

 5888/25724 [=====>........................] - ETA: 22s - loss: 0.6955 - acc: 0.5136
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 6912/25724 [=======>......................] - ETA: 21s - loss: 0.6959 - acc: 0.5107
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 7936/25724 [========>.....................] - ETA: 19s - loss: 0.6956 - acc: 0.5129
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 8960/25724 [=========>....................] - ETA: 19s - loss: 0.6954 - acc: 0.5156
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 9984/25724 [==========>...................] - ETA: 18s - loss: 0.6956 - acc: 0.5164
Epoch 00003: saving model to checkpoints/weights.03.hdf5
11008/25724 [===========>..................] - ETA: 16s - loss: 0.6962 - acc: 0.5160 ETA: 17s - 
Epoch 00003: saving model to checkpoints/weights.03.hdf5
12032/25724 [=============>................] - ETA: 15s - loss: 0.6969 - acc: 0.5151
Epoch 00003: saving model to checkpoints/weights.03

11776/25724 [============>.................] - ETA: 17s - loss: 0.6944 - acc: 0.5220
Epoch 00005: saving model to checkpoints/weights.05.hdf5
12800/25724 [=============>................] - ETA: 15s - loss: 0.6945 - acc: 0.5193
Epoch 00005: saving model to checkpoints/weights.05.hdf5
13824/25724 [===============>..............] - ETA: 14s - loss: 0.6952 - acc: 0.5176
Epoch 00005: saving model to checkpoints/weights.05.hdf5
14848/25724 [================>.............] - ETA: 13s - loss: 0.6953 - acc: 0.5189
Epoch 00005: saving model to checkpoints/weights.05.hdf5
15872/25724 [=================>............] - ETA: 11s - loss: 0.6952 - acc: 0.5207
Epoch 00005: saving model to checkpoints/weights.05.hdf5
16896/25724 [==================>...........] - ETA: 10s - loss: 0.6952 - acc: 0.5217
Epoch 00005: saving model to checkpoints/weights.05.hdf5
17920/25724 [===================>..........] - ETA: 9s - loss: 0.6956 - acc: 0.5192
Epoch 00005: saving model to checkpoints/weights.05.hdf5
18944/2

17664/25724 [===================>..........] - ETA: 8s - loss: 0.6949 - acc: 0.5227
Epoch 00007: saving model to checkpoints/weights.07.hdf5
18688/25724 [====================>.........] - ETA: 7s - loss: 0.6949 - acc: 0.5215
Epoch 00007: saving model to checkpoints/weights.07.hdf5
19712/25724 [=====================>........] - ETA: 6s - loss: 0.6948 - acc: 0.5220
Epoch 00007: saving model to checkpoints/weights.07.hdf5
20736/25724 [=======================>......] - ETA: 5s - loss: 0.6951 - acc: 0.5203
Epoch 00007: saving model to checkpoints/weights.07.hdf5
21760/25724 [========================>.....] - ETA: 4s - loss: 0.6949 - acc: 0.5210
Epoch 00007: saving model to checkpoints/weights.07.hdf5
22784/25724 [=========================>....] - ETA: 3s - loss: 0.6949 - acc: 0.5199
Epoch 00007: saving model to checkpoints/weights.07.hdf5
23840/25724 [==========================>...] - ETA: 2s - loss: 0.6950 - acc: 0.5186
Epoch 00007: saving model to checkpoints/weights.07.hdf5
24832/25724 [

23584/25724 [==========================>...] - ETA: 2s - loss: 0.6945 - acc: 0.5193
Epoch 00009: saving model to checkpoints/weights.09.hdf5
24576/25724 [===========================>..] - ETA: 1s - loss: 0.6946 - acc: 0.5194
Epoch 00009: saving model to checkpoints/weights.09.hdf5
25632/25724 [============================>.] - ETA: 0s - loss: 0.6945 - acc: 0.5196
Epoch 00009: saving model to checkpoints/weights.09.hdf5
25724/25724 [==============================] - 28s 1ms/sample - loss: 0.6945 - acc: 0.5196
Epoch 10/50
  928/25724 [>.............................] - ETA: 28s - loss: 0.6916 - acc: 0.5453
Epoch 00010: saving model to checkpoints/weights.10.hdf5
 1920/25724 [=>............................] - ETA: 27s - loss: 0.6955 - acc: 0.5318
Epoch 00010: saving model to checkpoints/weights.10.hdf5
 2944/25724 [==>...........................] - ETA: 26s - loss: 0.6941 - acc: 0.5262
Epoch 00010: saving model to checkpoints/weights.10.hdf5
 3968/25724 [===>..........................] - E

...

24704/25724 [===========================>..] - ETA: 1s - loss: 0.6953 - acc: 0.5198
Epoch 00040: saving model to checkpoints/weights.40.hdf5
25724/25724 [==============================] - 28s 1ms/sample - loss: 0.6953 - acc: 0.5195
Epoch 41/50
   32/25724 [..............................] - ETA: 32s - loss: 0.6653 - acc: 0.6562
Epoch 00041: saving model to checkpoints/weights.41.hdf5
 1024/25724 [>.............................] - ETA: 29s - loss: 0.6901 - acc: 0.5439
Epoch 00041: saving model to checkpoints/weights.41.hdf5
 2048/25724 [=>............................] - ETA: 27s - loss: 0.6914 - acc: 0.5337
Epoch 00041: saving model to checkpoints/weights.41.hdf5
 3072/25724 [==>...........................] - ETA: 25s - loss: 0.6949 - acc: 0.5156
Epoch 00041: saving model to checkpoints/weights.41.hdf5
 4096/25724 [===>..........................] - ETA: 23s - loss: 0.6969 - acc: 0.5115
Epoch 00041: saving model to checkpoints/weights.41.hdf5
 5120/25724 [====>.........................] -

 3840/25724 [===>..........................] - ETA: 23s - loss: 0.6956 - acc: 0.5094
Epoch 00043: saving model to checkpoints/weights.43.hdf5
 4864/25724 [====>.........................] - ETA: 22s - loss: 0.6939 - acc: 0.5193
Epoch 00043: saving model to checkpoints/weights.43.hdf5
 5888/25724 [=====>........................] - ETA: 21s - loss: 0.6931 - acc: 0.5211
Epoch 00043: saving model to checkpoints/weights.43.hdf5
 6912/25724 [=======>......................] - ETA: 20s - loss: 0.6939 - acc: 0.5184
Epoch 00043: saving model to checkpoints/weights.43.hdf5
 7936/25724 [========>.....................] - ETA: 19s - loss: 0.6948 - acc: 0.5137
Epoch 00043: saving model to checkpoints/weights.43.hdf5
 8960/25724 [=========>....................] - ETA: 18s - loss: 0.6950 - acc: 0.5144
Epoch 00043: saving model to checkpoints/weights.43.hdf5
 9984/25724 [==========>...................] - ETA: 16s - loss: 0.6948 - acc: 0.5168
Epoch 00043: saving model to checkpoints/weights.43.hdf5
11008/

 9728/25724 [==========>...................] - ETA: 19s - loss: 0.6933 - acc: 0.5244
Epoch 00045: saving model to checkpoints/weights.45.hdf5
10752/25724 [===========>..................] - ETA: 17s - loss: 0.6933 - acc: 0.5242
Epoch 00045: saving model to checkpoints/weights.45.hdf5
11776/25724 [============>.................] - ETA: 16s - loss: 0.6935 - acc: 0.5210
Epoch 00045: saving model to checkpoints/weights.45.hdf5
12800/25724 [=============>................] - ETA: 15s - loss: 0.6934 - acc: 0.5215
Epoch 00045: saving model to checkpoints/weights.45.hdf5
13824/25724 [===============>..............] - ETA: 13s - loss: 0.6938 - acc: 0.5181
Epoch 00045: saving model to checkpoints/weights.45.hdf5
14880/25724 [================>.............] - ETA: 12s - loss: 0.6937 - acc: 0.5190
Epoch 00045: saving model to checkpoints/weights.45.hdf5
15872/25724 [=================>............] - ETA: 11s - loss: 0.6939 - acc: 0.5177
Epoch 00045: saving model to checkpoints/weights.45.hdf5
16928/

15616/25724 [=================>............] - ETA: 11s - loss: 0.6949 - acc: 0.5235
Epoch 00047: saving model to checkpoints/weights.47.hdf5
16640/25724 [==================>...........] - ETA: 10s - loss: 0.6949 - acc: 0.5236
Epoch 00047: saving model to checkpoints/weights.47.hdf5
17664/25724 [===================>..........] - ETA: 9s - loss: 0.6949 - acc: 0.5224
Epoch 00047: saving model to checkpoints/weights.47.hdf5
18720/25724 [====================>.........] - ETA: 7s - loss: 0.6947 - acc: 0.5239
Epoch 00047: saving model to checkpoints/weights.47.hdf5
19712/25724 [=====================>........] - ETA: 6s - loss: 0.6946 - acc: 0.5238
Epoch 00047: saving model to checkpoints/weights.47.hdf5
20768/25724 [=======================>......] - ETA: 5s - loss: 0.6948 - acc: 0.5226
Epoch 00047: saving model to checkpoints/weights.47.hdf5
21760/25724 [========================>.....] - ETA: 4s - loss: 0.6952 - acc: 0.5200
Epoch 00047: saving model to checkpoints/weights.47.hdf5
22784/25724

21504/25724 [========================>.....] - ETA: 4s - loss: 0.6954 - acc: 0.5174
Epoch 00049: saving model to checkpoints/weights.49.hdf5
22528/25724 [=========================>....] - ETA: 3s - loss: 0.6953 - acc: 0.5178
Epoch 00049: saving model to checkpoints/weights.49.hdf5
23552/25724 [==========================>...] - ETA: 2s - loss: 0.6950 - acc: 0.5196
Epoch 00049: saving model to checkpoints/weights.49.hdf5
24576/25724 [===========================>..] - ETA: 1s - loss: 0.6949 - acc: 0.5205
Epoch 00049: saving model to checkpoints/weights.49.hdf5
25600/25724 [============================>.] - ETA: 0s - loss: 0.6951 - acc: 0.5193
Epoch 00049: saving model to checkpoints/weights.49.hdf5
25724/25724 [==============================] - 28s 1ms/sample - loss: 0.6951 - acc: 0.5192
Epoch 50/50
  928/25724 [>.............................] - ETA: 28s - loss: 0.6946 - acc: 0.5043
Epoch 00050: saving model to checkpoints/weights.50.hdf5
 1920/25724 [=>............................] - ETA

In [25]:
#Export our model to HDF5 file
nn.save("AlphabetSoupCharity_OPTIMIZATION3.h5")

In [26]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('AlphabetSoupCharity_OPTIMIZATION3.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 3s - loss: 0.7170 - acc: 0.4599
Loss: 0.717011364208366, Accuracy: 0.45994168519973755
